In [25]:
pip install ultralytics opencv-python numpy

Note: you may need to restart the kernel to use updated packages.


In [26]:
from ultralytics import YOLO

In [27]:
# Load custom-trained YOLOv11 model
try:
    model = YOLO("best.pt")
    print("YOLOv11 model loaded successfully!")
except Exception as e:
    print("Failed to load YOLOv11 model:", e)

YOLOv11 model loaded successfully!


In [28]:
import cv2

def detect_players(video_path, model):
    cap = cv2.VideoCapture(video_path)
    detections = []
    frame_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)

        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            if conf < 0.3:
                continue

            crop = frame[y1:y2, x1:x2]
            detections.append({
                "frame": frame_id,
                "bbox": (x1, y1, x2, y2),
                "crop": crop
            })

        frame_id += 1
    cap.release()
    return detections


In [29]:
import numpy as np

def get_histogram(crop):
    hsv = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1], None, [50, 60], [0, 180, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()


In [30]:
def match_players(detections_tacticam, detections_broadcast):
    mappings = {}

    # Compare each tacticam player to all broadcast players
    for i, tac in enumerate(detections_tacticam):
        best_match = None
        best_score = -1

        hist_tac = get_histogram(tac['crop'])

        for j, br in enumerate(detections_broadcast):
            hist_br = get_histogram(br['crop'])
            score = cv2.compareHist(hist_tac, hist_br, cv2.HISTCMP_CORREL)
            if score > best_score:
                best_score = score
                best_match = j

        mappings[i] = best_match
    return mappings


In [31]:
# Step 1: Detect
detections_broadcast = detect_players("broadcast.mp4", model)
detections_tacticam = detect_players("tacticam.mp4", model)


0: 384x640 3 players, 1595.4ms
Speed: 230.9ms preprocess, 1595.4ms inference, 129.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 624.4ms
Speed: 2.1ms preprocess, 624.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 603.0ms
Speed: 2.1ms preprocess, 603.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 682.9ms
Speed: 2.2ms preprocess, 682.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 758.8ms
Speed: 3.7ms preprocess, 758.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 players, 1 referee, 826.8ms
Speed: 2.9ms preprocess, 826.8ms inference, 4.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 goalkeeper, 12 players, 1 referee, 637.9ms
Speed: 2.5ms preprocess, 637.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 1 goalkeeper, 14 players, 1 refer

In [32]:
# Step 2: Match
player_mapping = match_players(detections_tacticam, detections_broadcast)

In [33]:
# Step 3: Display Mapping
print(" Player Mapping: tacticam broadcast")
for tid, bid in player_mapping.items():
    print(f"Tacticam player {tid} → Broadcast player {bid}")

 Player Mapping: tacticam broadcast
Tacticam player 0 → Broadcast player 497
Tacticam player 1 → Broadcast player 562
Tacticam player 2 → Broadcast player 33
Tacticam player 3 → Broadcast player 851
Tacticam player 4 → Broadcast player 633
Tacticam player 5 → Broadcast player 86
Tacticam player 6 → Broadcast player 75
Tacticam player 7 → Broadcast player 1493
Tacticam player 8 → Broadcast player 735
Tacticam player 9 → Broadcast player 1399
Tacticam player 10 → Broadcast player 739
Tacticam player 11 → Broadcast player 644
Tacticam player 12 → Broadcast player 630
Tacticam player 13 → Broadcast player 526
Tacticam player 14 → Broadcast player 657
Tacticam player 15 → Broadcast player 934
Tacticam player 16 → Broadcast player 1208
Tacticam player 17 → Broadcast player 630
Tacticam player 18 → Broadcast player 74
Tacticam player 19 → Broadcast player 1068
Tacticam player 20 → Broadcast player 1369
Tacticam player 21 → Broadcast player 1051
Tacticam player 22 → Broadcast player 403
Tactic